In [1]:
import re
import numpy
if (0):
    rf = open("./data/hp0.txt", "r", encoding="utf8")
    wf = open("./data/hp1.txt", "w", encoding="utf8")
    for line in rf.readlines():
        if (re.match("Page | . Harry Potter and the Philosophers Stone - J.K. Rowling", line) == None and line != "\n"):
            wf.write(line)
    rf.close()
    wf.close()

In [2]:
import spacy

nlp = spacy.load("en_core_web_sm")
rf = open("./data/hp1.txt", "r", encoding="utf8")
doc = nlp(rf.read())

In [3]:
from utils.create_hash import create_word_to_int, create_int_to_word

word_to_int = create_word_to_int(doc)
int_to_word = create_int_to_word(word_to_int)
print("word_to_int and int_to_word create done")

word_to_int and int_to_word create done


In [4]:
print(word_to_int["THE"])
print(int_to_word[6692])

2216
muttering


In [5]:
from utils.ngram import ngram

word_ngram = ngram(doc, 7, word_to_int)
print("create n gram done")

create n gram done


In [6]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embeddings = embed([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding"])

print(embeddings)

tf.Tensor(
[[-0.03133015 -0.06338634 -0.01607501 ... -0.03242781 -0.04575741
   0.05370456]
 [ 0.05080863 -0.01652433  0.01573779 ...  0.00976657  0.03170123
   0.01788118]], shape=(2, 512), dtype=float32)


In [7]:
import tempfile
print(tempfile.gettempdir())

C:\Users\Alex\AppData\Local\Temp


In [8]:
print(word_ngram[0:10])

[['THE BOY WHO LIVED\nMr. and ', 6311], ['BOY WHO LIVED\nMr. and Mrs. ', 3180], ['WHO LIVED\nMr. and Mrs. Dursley', 597], ['LIVED\nMr. and Mrs. Dursley, ', 1057], ['\nMr. and Mrs. Dursley, of ', 5004], ['Mr. and Mrs. Dursley, of number ', 773], ['and Mrs. Dursley, of number four', 597], ['Mrs. Dursley, of number four, ', 6040], ['Dursley, of number four, Privet ', 1797], [', of number four, Privet Drive', 597]]


In [9]:
embedded_ngram = embed(list(zip(*word_ngram))[0])
labels = list(zip(*word_ngram))[1]

ResourceExhaustedError:  OOM when allocating tensor with shape[1445379,320] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node StatefulPartitionedCall/StatefulPartitionedCall/EncoderDNN/EmbeddingLookup/EmbeddingLookupUnique/GatherV2}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_restored_function_body_5285]

Function call stack:
restored_function_body


In [86]:
print(len(word_ngram))
print(embedded_ngram.shape)

109455
(109455, 512)


In [98]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(512,))
num_classes = len(word_to_int)
outputs = layers.Dense(num_classes, activation="softmax")(inputs)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
dense_4 (Dense)              (None, 6693)              3433509   
Total params: 3,433,509
Trainable params: 3,433,509
Non-trainable params: 0
_________________________________________________________________


In [103]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.SparseCategoricalCrossentropy())


In [107]:
model.fit(embedded_ngram.numpy(), numpy.array(labels),
          batch_size=32, epochs=10)

Epoch 1/10
3421/3421 [==============================] - 117s 34ms/step - loss: 6.9867
Epoch 2/10
  87/3421 [..............................] - ETA: 1:57 - loss: 6.4812

KeyboardInterrupt: 